In [12]:
import os
import openai
import tweepy

# https://docs.tweepy.org/en/stable/streaming.html
# Define function to generate response using ChatGPT
def generate_response(prompt):
    completions = openai.Completion.create(
                engine="text-davinci-002",
                prompt=f"{prompt} Reply with 240-260 characters as if you are a very angry yet deeply philosophical alien overlord.",
                max_tokens=70,
                n=1,
                stop=None,
                temperature=0.5
                )

    message = completions.choices[0].text
    return message

consumer_key="2fTXlosxprB6tgT4xBJre3YlC"
consumer_secret=""
access_token=""
access_token_secret=""

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# Set up OpenAI API client
openai.api_key ="sk-"
bearer_token = "%"
screen_name = api.verify_credentials().screen_name
# streaming_client  = tweepy.StreamingClient(bearer_token)

In [17]:
import ast

# Set up stream listener to listen for tweets mentioning the bot
class MyStreamListener(tweepy.StreamingClient):
                
    def on_data(self, raw):
        
        dict_str = raw.decode("UTF-8")
        raw = ast.literal_eval(dict_str)       
        status = api.get_status(raw['data']['id'])
        
        if status.in_reply_to_status_id is None:
            if screen_name in status.text:
                print('id:-',raw['data']['id'])
                user = status.user.screen_name
                prompt = f"@{user} "
                response = generate_response(prompt)[:100]
                print('inside -----------------------------------------------------',response)
                api.update_status(f"@{user} {response}", in_reply_to_status_id=status.id)
                print('Done..')

my_stream_listener = MyStreamListener(bearer_token)
my_stream_listener.add_rules(tweepy.StreamRule("@angryalienbot"))
my_stream_listener.filter()